In [957]:
import numpy as np
from typing import Callable, List, Tuple, Union

# Solutions of Equations in One Variable

## Chapter 2.1 The Bisection Method
This method is derived by using _Extreme Value Theorem_ as indicated below

### Extreme Value Theorem
If $f\in C[a, b]$, then $c_1, c_2\in[a, b]$ exist with $f(c_1)\leq f(x)\leq f(c_2)$, for all $x\in[a, b]$. In addition, if $f$ is differentiable on $(a, b)$, then the numbers $c_1$ and $c_2$ occur either at the endpoints of $[a, b]$ or where $f'$ is zero.

Given $a, b$ WLOG let $f(a) < 0,f(b) > 0$, then by the _Extreme Value Theorem_ $\exists x\in[a, b]$ s.t. $f(a) < f(x) = 0 < f(b)$ since $f(a)<0<f(b)$. This means that $f(a)f(b) < 0 \implies\exists x\in[a, b]\text{ s.t. }f(x)=0$

In [958]:
def Bisection(func:Callable[[float], float], 
              left:float, 
              right:float, 
              iterations:int=30, 
              tol:float=1e-5)->List[float]:
    if func(left) * func(right) > 0:
        raise ValueError("Cannot apply Intermidiate Value Theorem")
    prev_p = np.inf 
    p_ = []
    for _ in range(iterations):
        p = left + (right - left) / 2
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
        if func(left) * func(p) < 0:
            right = p
        elif func(p) * func(right) < 0:    
            left = p
    return p_

## Chapter 2.2 Fixed-Point Iteration (FPI)
$$p_{n+1}=g(p)$$

In [959]:
def FPI(func:Callable[[float], float], 
        init:float, 
        iterations:int=30, 
        tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p = func(p)
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
    return p_

## Chapter 2.3 Newton's Method and its Extensions
### Newton's Method
$$p_n=p_{n-1}-\frac{f(p_{n-1})}{f'(p_{n-1})}$$

In [960]:
def Newton(f:Callable[[float], float], 
           df:Callable[[float], float], 
           init:float, 
           iterations:int=30, 
           tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p -= f(p) / df(p)
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
    return p_

### Secant Method
$$p_n=p_{n-1}-\frac{f(p_{n-1})(p_{n-1}-p_{n-2})}{f(p_{n-1})-f(p_{n-2})}$$

In [961]:
def Secant(f:Callable[[float], float], 
           p0:float, 
           p1:float, 
           iterations:int=30, 
           tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [p0, p1]
    p = p1
    for _ in range(iterations):
        prev_p = p
        p = p1 - (f(p1) * (p1 - p0) / (f(p1) - f(p0)))
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
        p0 = p1
        p1 = p
    return p_

### False Position
The method of _False Position_ generates approximations
in the same manner as the Secant method, but it includes a test to ensure that the root is
always bracketed between successive iterations (_Extreme Value Theorem_). Although it is not a method we generally
recommend, it illustrates how bracketing can be incorporated.

In [962]:
def FalsePosition(f:Callable[[float], float], 
                  p0:float, 
                  p1:float, 
                  iterations:int=30, 
                  tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [p0, p1]
    p = p1
    for _ in range(iterations):
        prev_p = p
        p = p1 - (f(p1) * (p1 - p0) / (f(p1) - f(p0)))
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
        if f(p) * f(p1) < 0:
            p0 = p1
        p1 = p
    return p_

In [963]:
import math

f = lambda x: x - math.cos(x)
g = lambda x: 1 + math.sin(x)

print(FalsePosition(f, 0.5, math.pi / 4, iterations=10))
print(Secant(f, 0.5, math.pi / 4, iterations=10))
print(Newton(f, g, math.pi / 4, iterations=10))

[0.5, 0.7853981633974483, 0.7363841388365822, 0.7390581392138897, 0.7390848638147098, 0.7390851305265789]
[0.5, 0.7853981633974483, 0.7363841388365822, 0.7390581392138897, 0.7390851493372764, 0.7390851332150645]
[0.7853981633974483, 0.7395361335152383, 0.7390851781060102, 0.739085133215161]


## Chapter 2.4 Error Analysis for Iterative Methods
### Order of Convergence
Suppose $\{p_n\}_n^\infty$ is a p that converges to $p$, with $p_n\neq p$ for all $n$. If positive constants $\lambda$ and $\alpha$ exist with

$$
\lim_{n\to\infty}\frac{|p_{n+1}-p|}{|p_n-p|^{\alpha}}=\lambda
$$
then $\{p_n\}_n^\infty$ __converges to__ $\textbf{p}$ __of order__ $\bf{\alpha}$, __with asymptotic error constant__ $\bf{\lambda}$

i) If $\alpha=1$ (and $\lambda < 1$), the p is __linearly convergent__.

ii) If $\alpha=2$ (and $\lambda < 1$), the p is __quadratically convergent__.

### Multiple Roots
A solution $p$ of $f(x)=0$ is a __zero of multiplicity__ $m$ of $f$ if for $x\neq p$, we can write $f(x)=(x-p)^mq(x)$, where $\lim_{x\to p}q(x)\neq0$. Those that have multiplicity one are identified __simple__ zeros.

The function $f\in C^1[a, b]$ has a simple zero at $p$ in $(a, b)$ iff $f(p)=0$ but $f'(p)\neq0$

If $f$ has a simple zero at $p$ then [Newton's Method](#newtons-method) converges at least quadratically.

The above statement can be generalized as
below

The function $f\in C^m[a, b]$ has a zero of multiplicity $m$ at $p$ in $(a, b)$ iff 
$$
0=f(p)=f'(p)=f''(p)=\cdots=f^{(m-1)}(p),\,\,\,\text{but}\,\,\,f^{(m)}(p)\neq0
$$

### Modified Newton's Method
To handle the problem of multiple roots of a function, __Modified Newton's Method__ is proposed as following:
$$
g(x)=x-\frac{f(x)f'(x)}{[f'(x)]^2-f(x)f''(x)}
$$

In [964]:
def ModifiedNewton(f:Callable[[float], float], 
                   df:Callable[[float], float], 
                   ddf:Callable[[float], float], 
                   init:float, 
                   iterations:int=30, 
                   tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p -= (f(p) * df(p)) / (df(p) ** 2 - f(p) * ddf(p))
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
    return p_

In [965]:
# Example 1 and 2
f = lambda x: np.exp(x) - x - 1
df = lambda x: np.exp(x) - 1
ddf = lambda x: np.exp(x)

Newton(f, df, 1, 5), ModifiedNewton(f, df, ddf, 1, 5)

([1,
  0.5819767068693265,
  0.31905504091081843,
  0.16799617288577048,
  0.08634887374778137,
  0.04379570367371408],
 [1,
  -0.23421061355351425,
  -0.00845827991076109,
  -1.1890183808588653e-05,
  -4.218590698935789e-11,
  -4.218590698935789e-11])

In [966]:
# Illustration when the case is simple zero
f = lambda x: x ** 3 + 4 * x ** 2 - 10
df = lambda x: 3 * x ** 2 + 8 * x
ddf = lambda x: 6 * x + 8
Newton(f, df, 1.5, 3), ModifiedNewton(f, df, ddf, 1.5, 3)

([1.5, 1.3733333333333333, 1.3652620148746266, 1.3652300139161466],
 [1.5, 1.3568989756979313, 1.3651958490280898, 1.3652300128418653])

## Chapter 2.5 Accelerating Convergence
### Aitken's $\Delta^2$ Method
Suppose $\{p_n\}_{n=0}^\infty$ is a linearly convergent p with limit $p$. To construct a p $\{\hat{p}_n\}_{n=0}^\infty$ that converges more rapidly to $p$, let us first assume that the signs of $p_n-p$, $p_{n+1}-p$, and $p_{n+2}-p$ agree and that $n$ is sufficiently large that

$$
\frac{p_{n+1}-p}{p_n-p}\approx\frac{p_{n+2}-p}{p_{n+1}-p}
$$

then

$$
p\approx\frac{p_{n+2}p_n-p_{n+1}^2}{p_{n+2}-2p_{n+1}+p_n}
$$

Adding the subtracting the terms $p^2_n$ and $2p_np_{n+1}$ in the numerator and grouping terms appropriately gives

$$

\begin{split}
p\approx\frac{p_np_{n+2}-2p_np_{n+1}+p_n^2-p_{n+1}^2+2p_np_{n+1}-p_{n}^2}{p_{n+2}-2p_{n+1}+p_n} \\
=\frac{p_n(p_{n+2}-2p_{n+1}+p_n)-(p_{n+1}^2-2p_np_{n+1}+p_{n}^2)}{p_{n+2}-2p_{n+1}+p_n} \\
=p_n-\frac{(p_{n+1}-p_n)^2}{p_{n+2}-2p_{n+1}+p_n}
\end{split}
$$

__Aitken's__ $\bf{\Delta^2}$ __method__ is based on assumption that the p $\{\hat{p}_n\}_{n=0}^\infty$, defined by
$$
\hat{p}_n=p_n-\frac{(p_{n+1}-p_n)^2}{p_{n+2}-2p_{n+1}+p_n}
$$
converges more rapidly to $p$ than the original p $\{p_n\}_{n=0}^\infty$

By introducing __forward difference__, $\Delta p_n$
$$
    \Delta p_n=p_{n+1}-p_n\,\,\,n\geq0
$$
$$
    \Delta^kp_n=\Delta(\Delta^{k-1}p_n)\,\,\,k\geq2
$$
The __Aitken's__ $\bf{\Delta^2}$ __method__ can be written as
$$
    \hat{p}_n=p_n-\frac{(\Delta p_n)^2}{\Delta^2p_n}
$$

where the p constructed as below:
$$
    p_0,\,\,p_1=g(p_0),\,\,p_2=g(p_1),\,\,\hat{p}_0=\{\Delta^2\}(p_0),\,\,p_3=g(p_2),\,\,\hat{p}_1=\{\Delta^2\}(p_1),\,\,\cdots
$$

### Steffensen's Method
There is another method, __Steffensen's Method__ which using the same formula but construct the p terms differently compared to  __Aitken's__ $\bf{\Delta^2}$ __method__. The construction is as below:

$$
p_0^{(0)},\,\,p_1^{(0)}=g(p_0^{(0)}),\,\,p_2^{(0)}=g(p_1^{(0)}),\,\,p_0^{(1)}=\{\Delta^2\}(p_0^{(0)}),\,\,p_1^{(1)}=g(p_0^{(1)})
$$

where $\Delta^2$ is the __Aitken's Operator__

In [967]:
def Aitken(f:Callable[[float], float], 
           init:float, 
           iterations:int=30, 
           tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p0 = init

    for _ in range(iterations):
        print(p0)
        prev_p = p0
        if _ % 3 == 0:
            p1 = f(p0)
            p2 = f(p1)
        elif _ % 3 == 1:
            p0 = p1
            p1 = p2
            p2 = f(p1)
        else:
            p1 = p0
            p0 = p2
            p2 = f(p1)
        p0 -= (p1 - p0) ** 2 / (p2 - 2 * p1 + p0)
        p_.append(p0)
        if abs(p0 - prev_p) < tol:
            return p_
    return p_

def Steffensen(f:Callable[[float], float], 
               init:float,
               iterations:int=30, 
               tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p1 = f(p)
        p2 = f(p1)
        p -= (p1 - p) ** 2 / (p2 - 2 * p1 + p)
        p_.append(p)
        if abs(prev_p - p) < tol:
            return p_
    return p_

def AitkenSeq(p:Union[List[Union[float, int]], np.ndarray]):
    n = len(p)
    if n < 3:
        raise ValueError("p length must be at least 3")
    
    p_hat = [0] * (n - 2)
    for idx in range(n - 2):
        p_hat[idx] = p[idx] - ((p[idx + 1] - p[idx]) ** 2) / (p[idx + 2] - 2 * p[idx + 1] + p[idx])
    return p_hat

In [968]:
# Illustration
f = lambda x: np.sqrt(10 / (x + 4))
Steffensen(f, 1.5, 3)

[1.5, 1.3652652239572602, 1.3652300134165856, 1.3652300134140969]

In [969]:
# Exercise 1
f = lambda x: (2 - np.exp(x) + x ** 2) / 3
print(Aitken(f, 0.5, 6))
f = lambda x: np.sqrt(np.exp(x) / 3)
print(Aitken(f, 0.75, 6))
f = lambda x: 3 ** (-x)
print(Aitken(f, 0.5, 6))
f = lambda x: np.cos(x)
print(Aitken(f, 0.5, 6))

0.5
0.2586844275657909
0.2576132107157495
0.2575114164710768
0.25753028544849726
[0.5, 0.2586844275657909, 0.2576132107157495, 0.2575114164710768, 0.25753028544849726, 0.2575302854404426]
0.75
0.9078585524534872
0.9095675068671815
0.9098116422269557
0.9100075688435889
[0.75, 0.9078585524534872, 0.9095675068671815, 0.9098116422269557, 0.9100075688435889, 0.9100075717340753]
0.5
0.5481009645113577
0.5479150736907433
0.5477417510788966
0.5478086222095319
[0.5, 0.5481009645113577, 0.5479150736907433, 0.5477417510788966, 0.5478086222095319, 0.5478086218552766]
0.5
0.7313851863825818
0.7360866917130169
0.740750412874417
0.7390847204845993
[0.5, 0.7313851863825818, 0.7360866917130169, 0.740750412874417, 0.7390847204845993, 0.7390849457924547]


In [970]:
# Exercise 11
f = lambda x: (2 - np.exp(x) + x ** 2) / 3
print(Steffensen(f, 0., 6))
f = lambda x: 0.5 * (np.sin(x) + np.cos(x))
print(Steffensen(f, 0., 6))
f = lambda x: np.sqrt(np.exp(x) / 3)
print(Steffensen(f, 0.25, 6))
f = lambda x: 5 ** (-x)
print(Steffensen(f, 0.3, 6))

[0.0, 0.25950408123897534, 0.2575303797852654, 0.257530285439861]
[0.0, 0.7776147730392706, 0.7048722519451539, 0.7048120020784795, 0.7048120020012982]
[0.25, 0.8840887418688467, 0.909945070754703, 0.9100075721177387, 0.9100075724887092]
[0.3, 0.47832613137585256, 0.469641613090862, 0.46962192303711814, 0.46962192293561056]


In [971]:
# Exercise 13
p = 1 / np.arange(1, 10)
print(AitkenSeq(p))
p = 1 / np.arange(1, 10) ** 2
print(AitkenSeq(p))

# Exercise 17
taylor_poly = lambda n: 1 + sum((np.exp(0) / math.factorial(i)) for i in range(1, n + 1))

p = [taylor_poly(idx) for idx in range(11)]
print(AitkenSeq(p))

[0.24999999999999978, 0.16666666666666674, 0.12500000000000003, 0.09999999999999978, 0.08333333333333347, 0.07142857142857154, 0.06249999999999978]
[0.07954545454545459, 0.036324786324786335, 0.0206117021276596, 0.013243243243243233, 0.009215991692627188, 0.006779424098406497, 0.005194698952879568]
[3.0, 2.750000000000001, 2.7222222222222223, 2.71875, 2.7183333333333337, 2.718287037037037, 2.71828231292517, 2.7182818700396827, 2.7182818317656285]


## Chapter 2.6 Zeros of Polynomials and Muller's Method
### Horner's Method
Let
$$ P(x) = a_nx^n+a_{n-1}+x^{n-1}+\cdots+a_1x+a_0 $$
Define $b_n=a_n$ and
$$ b_k=a_k+b_{k+1}x_0,\,\,\,\text{for }k=n-1, n-2\cdots,1, 0$$
Then $b_0=P(x_0)$.Moreover, if
$$ Q(x)=b_nx^{n-1}+b_{n-1}x^{n-2}+\cdots+b_2x+b_1$$
Then 
$$ P(x) = (x-x_0)Q(x)+b_0 $$
Cooperate with _Newton's Method_ to find the zeros of polynomials.

In [972]:
def Horner(coef:Union[np.ndarray, List[Union[int, float]]], 
           x0:Union[float, int]) -> List[Union[float, int, np.ndarray]]:
    coef_ = np.copy(coef).astype(float)
    f_ = np.zeros(len(coef_) - 1, dtype=float)
    f = coef_[-1]
    fp = coef_[-1]
    f_[-1] = coef_[-1]
    for jdx in range(len(coef_) - 2, 0, -1):
        f = x0 * f + coef_[jdx]
        f_[jdx - 1] = f
        fp = x0 * fp + f
    f = x0 * f + coef_[0]
    # f_[0] = f
    return [f, fp, f_]

In [973]:
Horner([-4, 3, -3, 0, 2], -2)

[10.0, -49.0, array([-7.,  5., -4.,  2.])]

In [974]:
import cmath
def HornerNewton(coef:Union[np.ndarray, List[Union[int, float]]], 
                 x0:Union[float, int],
                 iterations:int=100,
                 tol:float=1e-6) -> List[Union[float, int]]:
    p = x0
    roots = []
    coef_ = np.copy(coef).astype(float)
    for _ in range(len(coef) - 1):
        if len(coef_) == 3:
            break
        # Newton's Method to approximate zeros
        prev_p = np.inf
        for _ in range(iterations):
            prev_p = p
            f, fp, _ = Horner(coef_, p)
            p -= f / fp
            if np.abs(prev_p - p) < tol:
                break
        # Horner's Method to divide the original polynomial
        # by the approximated zero
        f, fp, coef_ = Horner(coef_, p)
        roots.append(p)
    r1 = (-coef_[1] + cmath.sqrt(coef_[1] ** 2 - 4 * coef_[0] * coef_[2])) / (2 * coef_[2])
    r2 = (-coef_[1] - cmath.sqrt(coef_[1] ** 2 - 4 * coef_[0] * coef_[2])) / (2 * coef_[2])
    roots.extend([r1, r2])
    return roots

In [975]:
HornerNewton([-5040, 1602, 1127, -214, -72, 4, 1], 8)

[7.0,
 3.0000000000000004,
 2.0000000000000013,
 -3.0000000000000107,
 (-4.999999999999992+0j),
 (-7.999999999999997+0j)]

In [976]:
HornerNewton([-4, 3, -3, 0, 2], -2)

[-1.738956256451892,
 1.2548818848342944,
 (0.24203718580879874+0.926245487267532j),
 (0.24203718580879874-0.926245487267532j)]

### Muller's Method
Müller’s method is similar to the
Secant method. But whereas the
Secant method uses a line
through two points on the curve
to approximate the root, Müller’s
method uses a parabola through three points on the curve for the approximation.

In [977]:
import cmath
Muller_Params = Union[complex, float]
def Muller(f:Callable[[Muller_Params], float], 
           p0:Muller_Params, p1:Muller_Params, p2:Muller_Params, 
           iterations:int=20, tol:float=1e-5) -> Muller_Params:
    for _ in range(iterations):
        h1 = p1 - p0
        h2 = p2 - p1
        delta1 = (f(p1) - f(p0)) / h1
        delta2 = (f(p2) - f(p1)) / h2
        d = (delta2 - delta1) / (h2 + h1)
        b = delta2 + h2 * d
        D = cmath.sqrt(b ** 2 - 4 * f(p2) * d)
        denoms = [b + D, b - D]
        if np.abs(h := -2 * f(p2)) < tol:
            return p
        p = p2 + h / max(denoms, key=abs)
        p0, p1, p2 = p1, p2, p
    return p

In [978]:
f = lambda x: x ** 4 - 3 * x ** 3 + x ** 2 + x + 1
print(Muller(f, 0.5, -0.5, 0))
print(Muller(f, 0.5, 1, 1.5))
print(Muller(f, 1.5, 2, 2.5))

(-0.33909283338402885+0.44663010055724217j)
(1.389389619617081+0j)
(2.288794993947653+0j)
